In [1]:
abs(-10)

10

In [2]:
x = abs(-10)

In [3]:
x

10

这是函数调用，还不是函数式，而我们把函数赋值给变量

In [4]:
fx = abs

In [5]:
fx

<function abs>

变量指向函数

调用

In [6]:
fx(-11)

11

## 高阶函数的应用：

In [8]:
#x,y,f，将规则f传入函数，并且将x和y分别计算规则后相加
def applyFunc(x,y,f):
    return f(x)+f(y)
applyFunc(-10,-20,abs)

30

map/reduce函数

In [11]:
#map有两个参数，一个是元素处理规则函数，另一个是要处理的Iterable可迭代的集合
def pow2(x):
    return x * x

sourceList = [1,2,3,4,5,6,7,8,9,10]
afterList = map(pow2, sourceList)
print(list(afterList))
#然后对应映射处理

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


## 高阶函数将运算抽象了，可以自己传入运算规则来执行map
list(map(str,sourceList))

In [18]:
#reduce化简结果函数
from functools import reduce
def toSum(x,y):
    return x + y
reduce(toSum, sourceList)

55

## 过滤序列，筛选函数

In [20]:
#判断是否为奇数
def is_odd(n):
    return n % 2 == 1
#第一个参数是过滤条件函数，会传入每一项然后判断返回值是否true，来决定是否保留并返回
list(filter(is_odd,sourceList))

[1, 3, 5, 7, 9]

In [21]:
#删除掉序列中的空字符串
def not_empty(s):
    return s and s.strip()

list(filter(not_empty, ['A', '', 'B', None, 'C', '  ']))

['A', 'B', 'C']

In [26]:
tumblelist = [-5,7,-1,4,11,-13]
#reverse是否反转结果
sorted(tumblelist,  key = abs, reverse = True)

[-13, 11, 7, -5, 4, -1]

In [25]:
sorted(['bob', 'about', 'Zoo', 'Credit'])

['Credit', 'Zoo', 'about', 'bob']

函数作为返回值

In [27]:
def calc_sum(*nums):
    sum = 0
    for x in nums:
        sum = sum + x
    return sum

In [37]:
#这个函数时上个函数的替代品，是返回函数而不是直接返回结果
def calc_sum_lazy(*nums):
    def calc_sum():
        sum = 0
        for x in nums:
            sum = sum + x
        return sum
    return calc_sum


In [39]:
doSum = calc_sum_lazy(1,2,3,4,5)
print(doSum())

15


## 闭包

In [40]:
def count():
    fs = []
    for i in range(1, 4):
        def f():
             return i*i
        fs.append(f)
    return fs

f1, f2, f3 = count()


In [43]:
f1()

9

In [44]:
f2()

9

In [45]:
f3()

9

由于def f()函数定义了一个闭包，变量i就成了不可变引用，那么每次引用的i事实上是range结束后
最后一个值
Python由于所有的类型都是引用类型，一切都是指针，所以三个函数中的i指向的值其实就是一个值，也
也就是range最后一个值3，最后f()中i*i得到的值便都是9，因此，小心不要掉到这个指针陷阱里。

lambda结构表达式，匿名函数，闭包
但是可惜，lambda表达式只允许一行表达式且有返回值，可以赋值

事实上一般可以将函数赋值给变量来实现延后调用。

In [49]:
#例如：
def pow2(x):
    return x * x


In [50]:
#转成匿名函数
f = lambda x : x * x
f(9)

81

In [54]:
#或者将匿名函数作为返回值返回
def pow2sum(x,y):
    return lambda: x * x + y * y
calcp2s = pow2sum(3,4)
calcp2s()

25

In [56]:
#函数默认属性
pow2sum.__name__

'pow2sum'

In [61]:
#装饰器函数log，接收函数作为参数，并且返回另一个函数，这个函数惰性处理了被修饰的函数，在运行
#运行
def log(func):
    def wrapper(*args,**kw):
        print('调用函数: %s' % func.__name__)
        return func(*args,**kw)
    return wrapper

In [58]:
@log 
def sum(x,y):
    return x*x+y*y

In [60]:
sum(1,3)

调用函数: sum


10

In [63]:
#如果装饰器需要参数，可以重新定义参数
def logStr(text):
    def deco(func):
        def wrapper(*args,**kw):
            print('%s: %s' % (text, func.__name__))
            return func(*args,**kw)
        return wrapper
    return deco

In [66]:
@logStr('执行') 
def sum(x,y):
    return x * x + y * y

In [65]:
sum(1,2)

执行: sum


5

In [73]:
#三层嵌套的效果
sum = logStr('CE')(sum)

In [74]:
sum(1,3)

CE: wrapper
CE:: wrapper
调用函数: wrapper
执行: sum


10

In [75]:
sum.__name__


'wrapper'

In [79]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

def logStr(text):
    def decorator(func):
        @functools.wraps(func) #说明func传入的函数是修饰到wrapper上的，可以矫正函数的元数据
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator


In [80]:
@logStr('执行') 
def sum(x,y):
    return x * x + y * y

In [81]:

sum.__name__

'sum'


## 偏函数

int函数是将字符串转换成int类型, base参数表示进位制

In [82]:
int('10010',base=2)

18

In [85]:
#每次都传base=2嫌麻烦，不如写个默认参数，每次省掉base=2
def int2bin(x,base=2):
    return int(x,base)

In [89]:
int2bin('1000000')

64

In [90]:
int2bin('101001')

41

但这些都不是偏函数，因为我们是定义了新的函数。而我们可以通过偏函数进行柯里化
用柯里化以后的指向函数的变量实现这个功能


In [92]:
import functools
int2 = functools.partial(int,base=2)
int2('10000')

16